In [7]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
from random import uniform
import matplotlib.pyplot as plt

In [8]:
class Neuron:
    'Neuron is a superclass that defines the global and default neuron\'s behaviour'
    
    def __init__(self, dimension, learning_rate):
        """Neuron Constructor.
            #self: means an object scope method, not passed by programmer, but by the interpreter
            @dimension: size of dimension (for XOR it is 2 entries/inputs)
            @output_nodes: an int amount of neurons nodes that process the data & retrieve the result
            @learning_rate: the rate/size of the learning/training process (it's as lower as good)
        """
        
        # Set attrs
        self.dimension = dimension
        self.lrate = learning_rate
        
        # Init empty vars that will be updated
        self.out = None
        self.error = None
        
        # Init the aux variables
        self.weights = [uniform(0,1) for x in range(dimension)]
        self.bias = uniform(0, 1)
       
    def activation(self, fx):
        return 1/(1 + np.exp(-fx))
    
    def update(self, inputs):
        """update the weights & it bias
            @input = matrix with the train inputs
        """

        counter = 0
        for i in inputs:
            delta = self.lrate * self.error
            self.weights[counter] -= (delta*i)
            self.bias -= delta
            counter+=1

    def feedforward(self, inputs):
        counter = 0
        sum = self.bias
        for i in inputs:
            sum += i * self.weights[counter]
            counter += 1
        self.out = self.activation(sum)
        
    def backward(self):
        """
            Defined on it children
        """
        pass        

In [9]:
class HiddenNeuron(Neuron):
    'HiddenNeuron is a child of Neuron that receives feedforward from input layer and the backprop from the Output'
    
    def __init__(self, dim, lrate=0.2):
        super(HiddenNeuron, self).__init__(dim, lrate)

    def backward(self, deltas, weights):
        sum = 0
        size = len(deltas)
        for x in range(size):
            sum += deltas[x] * weights[x]
        self.error = self.out * (1 - self.out) * sum

In [10]:
class OutputNeuron(Neuron):
    'OutputNeuron is a child of Neuron that receives feedforward from hidden layer, backprop to hidden & retrieves it final result'
    def __init__(self, dim, lrate=0.2):
        super(OutputNeuron, self).__init__(dim, lrate)

    def backward(self, target):
        self.error = self.out * (1 - self.out) * (self.out - target)

In [11]:
class Model:
    'Model is the way  Optional class documentation string'
    
    def __init__(self):
        self.hidden = [HiddenNeuron(2) for i in range(2)]
        self.output = OutputNeuron(2)

    def predict(self, input):
        temp = []
        for x in range(2):
            self.hidden[x].feedforward(input)
            temp.append(self.hidden[x].out)
        self.output.feedforward(temp)
        return self.output.out

    def train(self, inputs, targets, epochs):
        it = 0
        i = 0
        size = len(inputs)
        while it < epochs:
            if i == size:
                i = 0
            feature = inputs[i]
#             print('Feature: ' + str(feature))
#             print('Output weights : ' + str(self.output.weights))
#             print('Hidden 1 weights : ' + str(self.hidden[0].weights))
#             print('Hidden 2 weights : ' + str(self.hidden[1].weights))
            temp = []
            for x in range(2):
                self.hidden[x].feedforward(feature)
                temp.append(self.hidden[x].out)
            self.output.feedforward(temp)
            self.output.backward(targets[i])
            deltas = []
            deltas.append(self.output.error)
            weights = []
            weights.append([self.output.weights[0]])
            weights.append([self.output.weights[1]])
            for x in range(2):
                self.hidden[x].backward(deltas, weights[x])
            for x in range(2):
                self.hidden[x].update(feature)
            self.output.update(temp)
            it += 1
            i += 1

In [12]:
inputs = [[0,0], [0,1], [1,0], [1,1]]
outputs = [0, 1, 1, 0]
epochs = 100000

m = Model()

m.train(inputs, outputs, epochs)

for i in inputs:
    p = m.predict(i)
    print(str(i) + ' => ' + str(p))

[0, 0] => 0.02169601206075658
[0, 1] => 0.9801826926686691
[1, 0] => 0.980095178172251
[1, 1] => 0.02244996213016803
